In [62]:
import pandas as pd
import numpy as np
import mysql.connector
import os
from mysql.connector import Error
from datetime import datetime, date, timedelta
import ConeccionesSucursales as cs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [63]:
#Definimos la ruta en donde queremos guardar los archivos
ruta_guardar = '/home/estadistico/Documents/Erick/Reportes diarios'
ruta_asig_baz = '/home/estadistico/Documents/Erick/Banco Azteca/Asignacion csv'
ruta_pagos = '/home/estadistico/Documents/Erick'
mes = 20210101
mes2 = 'Enero2021'
mes3 = pd.to_datetime('01/01/2021',dayfirst=True)
prim_sem_mes = 1

In [64]:
#Agregamos las variables a ocupar
servidor = '192.168.15.12'
puerto = int('3306')
usuario = 'estadisticas'
contraseña = 'estadisticas8474'
base = 'procesos_externos'

In [65]:
#Lectura pagos Bradesco
pagos_brad = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Bradesco/Pagos/Acumulado de pagos enero de 2021.xlsx')
pagos_brad = pagos_brad.iloc[:,0:9]
pagos_brad = pagos_brad.drop_duplicates()
pagos_brad2 = pd.pivot_table(pagos_brad,index=['NUMERO_TARJETA','FECHA_PAGO','TIPO_ASIGANCION'],values=['MONTO'],aggfunc=np.sum)
pagos_brad2 = pd.DataFrame(pagos_brad2.to_records())
pagos_brad2 = pagos_brad2.loc[pagos_brad2['FECHA_PAGO']>=mes3]

In [66]:
#Lectura pagos Liverpool
sheet_liv = ['SA002REM','SA001REM','DKN04REM','DKN02REM','DKN00REM']
pagos_liv = pd.DataFrame()
for i in sheet_liv:
    pagos_liv_new = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Liverpool/Pagos/PAGOS ENERO2021.xlsx',sheet_name = str(i),converters={'Cuenta':str})
    pagos_liv = pd.concat([pagos_liv,pagos_liv_new],ignore_index=True)
pagos_liv = pagos_liv.drop_duplicates()
pagos_liv = pagos_liv.iloc[:,0:6]
pagos_liv.columns = ['Id','Cuenta','Fecha Prom','Fecha Pago','Monto','Posteo']
try: 
    pagos_liv['Fecha Pago'] = pd.to_datetime(pagos_liv['Fecha Pago'],dayfirst=True)
except: pagos_liv['Fecha Pago']   
pagos_liv['Fecha Pago'] = [datetime.strftime(x,'%d/%m/%Y') for x in pagos_liv['Fecha Pago']]
try: 
    pagos_liv['Fecha Prom'] = pd.to_datetime(pagos_liv['Fecha Prom'],dayfirst=True)
except: pagos_liv['Fecha Prom']    
pagos_liv['Fecha Prom'] = [datetime.strftime(x,'%d/%m/%Y') for x in pagos_liv['Fecha Prom']]
pagos_liv2 = pd.pivot_table(pagos_liv,index=['Id','Cuenta','Fecha Pago','Fecha Prom'],values=['Monto'],aggfunc=np.sum)
pagos_liv2 = pd.DataFrame(pagos_liv2.to_records())
#pagos_liv2 = pagos_liv2.loc[pagos_liv2['Fecha Pago']>=mes3]

In [67]:
#Lectura pagos Credifiel
sheet_cred = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Pagos/')
pagos_cred = pd.DataFrame()
for j in sheet_cred:
    pagos_cred_new = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Pagos/'+j+'')
    pagos_cred = pd.concat([pagos_cred,pagos_cred_new],ignore_index=True)
#pagos_cred = pagos_cred.iloc[:,[12,1,8]]
pagos_cred2 = pd.pivot_table(pagos_cred,index=['ID','FECHA'],values=['IMPORTE'],aggfunc=np.sum)
pagos_cred2 = pd.DataFrame(pagos_cred2.to_records())

In [68]:
#Lectura pagos Banco Azteca
#sheet_baz = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/')
#pagos_baz = pd.DataFrame()
#for k in sheet_baz:
#    pagos_baz_new = pd.read_csv(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/'+k+'')
#    pagos_baz = pd.concat([pagos_baz,pagos_baz_new],ignore_index=True)
#pagos_baz = pagos_baz.drop_duplicates()
#pagos_baz = pagos_baz.loc[pagos_baz['FNMONTOPAGADO']>0]
#pagos_baz['FDFECHAABONO'] = pd.to_datetime(pagos_baz['FDFECHAABONO'],dayfirst=True)
#pagos_baz = pagos_baz.loc[pagos_baz['FDFECHAABONO']>=mes3]
#pagos_baz2 = pd.pivot_table(pagos_baz,index=['CAMPANAID','FDFECHAABONO','FIIDPERIODO','FNSCOMPROMISO','FCEMPNUMCORTE','FNMONTOPROMETIDO'],values=['FNMONTOPAGADO'],aggfunc=np.sum)
#pagos_baz2 = pd.DataFrame(pagos_baz2.to_records())

In [69]:
#Leemos facturacion Banco Azteca
#sheet_baz2 = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Facturacion/')
#fact_baz = pd.DataFrame()
#for k in sheet_baz2:
#    fact_baz_new = pd.read_csv(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Facturacion/'+k+'',encoding='latin-1')
#    fact_baz = pd.concat([fact_baz,fact_baz_new],ignore_index=True)
#fact_baz = fact_baz.drop_duplicates()
#fact_baz = fact_baz.loc[fact_baz['Número Semana']>=prim_sem_mes]
#fact_baz['Final de Semana'] = pd.to_datetime(fact_baz['Final de Semana'],dayfirst=True)
#fact_baz['Inicio de Semana'] = pd.to_datetime(fact_baz['Inicio de Semana'],dayfirst=True)

In [70]:
#Asignamos valores a los parametros \n"
today = date.today().strftime('%Y%m%d')
now = datetime.now().strftime('%d-%m-%Y %H-%M')
hoy = np.datetime64('today')
ayer = np.datetime64(date.today() - timedelta(1))
today2 = date.today().strftime('%d-%m-%Y')

In [71]:
#Hacemos la conexion con el servidor\n",
try:
    conn = mysql.connector.connect(user=usuario,
                               password=contraseña,
                               host=servidor,
                               port=3306,
                               database=base)
    conn.set_charset_collation('latin1')
except mysql.Error as e:
    print("Failed to execute stored procedure: {}".format(error))

cursor = conn.cursor()

In [72]:
#Hacemos la asignacion de cada una de las metas para cambiar de arancel sin tomar el 0 en cuenta
c04pv1, arancel_4pv1 = 1204704, .07
c04pv2, arancel_4pv2 = 1807056, .09
c04pv3, arancel_4pv3 = 2409407, .12
c05pv1, arancel_5pv1 = 662269, .09
c05pv2, arancel_5pv2 = 993403, .10
c05pv3, arancel_5pv3 = 1324538, .14
c06pv1, arancel_6pv1 = 807172, .10
c06pv2, arancel_6pv2 = 1210759, .11
c06pv3, arancel_6pv3 = 1614345, .16
a1, arancel_a1 = 260190, .27
a2, arancel_a2 = 390285, .30
a3, arancel_a3 = 520381, .33
b1, arancel_b1 = 215083, .25
b2, arancel_b2 = 322624, .28
b3, arancel_b3 = 430165, .31
c11, arancel_c11 = 1430989, .17
c12, arancel_c12 = 2146483, .20
c13, arancel_c13 = 2861978, .23
c21, arancel_c21 = 626634, .19
c22, arancel_c22 = 939951, .22
c23, arancel_c23 = 1253267, .25
c31, arancel_c31 = 246425, .21
c32, arancel_c32 = 369637, .24
c33, arancel_c33 = 492849, .27
c41, arancel_c41 = 123529, .23
c42, arancel_c42 = 185294, .26
c43, arancel_c43 = 247059, .29

In [73]:
#Funcion para hacer la consulta sql, ya sea de gestion o de asignacion en donde requerimos tres parametros, la tabla de donde vamos a sacar la consulta, en caso de asignacin requerimos los otros dos parametros fecha inicial y fecha final
def consulta_sql(table,from_date=None,to_date=None):
    if from_date is None:
        sql_mes = cursor.callproc(table,)
        for result in cursor.stored_results():
            sql_mes = pd.DataFrame(result.fetchall())
    else:
        sql_mes = cursor.callproc(table, [from_date,to_date])
        for result in cursor.stored_results():
            sql_mes = pd.DataFrame(result.fetchall())
    sql_mes = sql_mes.drop_duplicates()
    return sql_mes

In [74]:
#Funcion para transformar varias columnas a numero
def transform_numeric(df,cols):
    return df[cols].apply(pd.to_numeric, errors='coerce')

#Funcion que convierte a dataframe y rellena espacios con 0
def to_dataframe(df):
    df = pd.DataFrame(df.to_records())
    df.fillna(0,inplace=True)
    return df

# Reporte Liverpool

In [75]:
#Hacemos la consulta referente a las gestiones de Liverpool
gestion_liv_mes = cs.query_naucalpan('liverpool_rpt_gestiones_detallado',mes,today)
gestion_liv_mes.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','fecha_promesa','monto_promesa','comentarios']
gestion_liv_mes = gestion_liv_mes.loc[gestion_liv_mes['dictamen']=='PROMESA']
gestion_liv_mes['anio'] = [int(x[:4]) for x in gestion_liv_mes['fecha_promesa']]
gestion_liv_mes = gestion_liv_mes.loc[(gestion_liv_mes['anio']>2010)&(gestion_liv_mes['anio']<2100)]
gestion_liv_mes['fecha_promesa'] = pd.to_datetime(gestion_liv_mes['fecha_promesa'],dayfirst=False)

In [76]:
#Asignacion Liverpool
asignacion_liv = pd.read_csv(''+ruta_pagos+'/Liverpool/Asignacion/Asignaciones csv/Asignacion '+mes2+'.csv',converters={'# CUENTA':str})
asignacion_liv = asignacion_liv[['ID AGENCIA', '# CUENTA']]
asignacion_liv.columns = ['division','credito']
asignacion_liv.head(1)

,division,credito
0,SA002REM,0001400000007731973


In [77]:
#Hacemos el consolidado
consolidado_liv_mes = pd.merge(gestion_liv_mes,asignacion_liv,how='left',on=['credito'])
cols = ['monto_promesa']
consolidado_liv_mes[cols] = consolidado_liv_mes[cols].apply(pd.to_numeric, errors='coerce')
consolidado_liv_mes = consolidado_liv_mes[(consolidado_liv_mes['monto_promesa']>50)]
consolidado_liv_mes = consolidado_liv_mes[(consolidado_liv_mes['monto_promesa']<500000)]
consolidado_liv_mes['division'].fillna('COBRANZA',inplace=True)
consolidado_liv_mes['division'] = consolidado_liv_mes['division'].replace('','COBRANZA')
consolidado_liv_mes = consolidado_liv_mes.drop_duplicates(subset = ['usuario','credito','fecha_promesa','monto_promesa'])

In [78]:
productos = [('DKN00REM','pren'),('DKN02REM','REM2'),('DKN04REM','REM4'),('SA001REM','SB02'),('SA002REM','REM9')]
productos = pd.DataFrame(productos)
productos.columns = ['id','division']
consolidado_liv_mes = pd.merge(consolidado_liv_mes,productos,how='left',on=['division'])

In [79]:
#Asignamos los aranceles de liverpool
aranceles = [('DKN00REM',0.28),('DKN02REM',0.4),('DKN04REM',0.5),('SA001REM',0.15),('SA002REM',0.16)]
aranceles = pd.DataFrame(aranceles)
aranceles.columns = ['Id','Arancel']

#Agregamos aranceles al consolidado
pagos_liv3 = pd.merge(pagos_liv2,aranceles,how='left',on='Id')
pagos_liv3['PagoArancel'] = [x*y for x,y in zip(pagos_liv3['Monto'],pagos_liv3['Arancel'])]

In [80]:
#Unimos el consolidado con los pagos 
consolidado_pagos_liv = pd.merge(consolidado_liv_mes,pagos_liv3,how='left',left_on=['credito'],right_on=['Cuenta'])
consolidado_pagos_liv = consolidado_pagos_liv.drop_duplicates(subset = ['credito','Monto','usuario','fecha_promesa'])
consolidado_pagos_liv['Fecha Pago'] = pd.to_datetime(consolidado_pagos_liv['Fecha Pago'],dayfirst=True)
consolidado_pagos_liv['dia_prom'] = consolidado_pagos_liv['fecha_promesa'].dt.day
consolidado_pagos_liv['dia_pago'] = consolidado_pagos_liv['Fecha Pago'].dt.day
cols = ['dia_prom','dia_pago','Monto']
consolidado_pagos_liv[cols] = consolidado_pagos_liv[cols].apply(pd.to_numeric, errors='coerce',downcast='integer')
consolidado_pagos_liv['dia_pago'].fillna(0,inplace=True)
try:
    consolidado_pagos_liv['dia_pago'] = [int(x) for x in consolidado_pagos_liv['dia_pago']]
except: consolidado_pagos_liv['dia_pago'] = consolidado_pagos_liv['dia_pago']

In [81]:
#Condicionamos a los gestores cuyas promesas y pagos estan en un plazo de 3 dias
consolidado_pagos_liv['PagoReal'] = [np.where((z < x+pd.Timedelta(3, unit='D')) & ((z > x-pd.Timedelta(3, unit='D') )), y, 0) for x,y,z in zip(consolidado_pagos_liv['fecha_promesa'],consolidado_pagos_liv['Monto'],consolidado_pagos_liv['Fecha Pago'])]
consolidado_pagos_liv['PagoReal'] = [x.item() for x in consolidado_pagos_liv['PagoReal']]

In [82]:
consolidado_pagos_liv['MontoArancel'] = [x*y for x,y in zip(consolidado_pagos_liv['PagoReal'],consolidado_pagos_liv['Arancel'])]

In [83]:
#Obtenemos datos de los gestores de Liverpool
promesas_mes_liv = pd.pivot_table(consolidado_pagos_liv,index=['division'],values=['monto_promesa'],aggfunc=['count',np.sum],margins=True,dropna=False)
promesas_mes_liv = pd.DataFrame(promesas_mes_liv.to_records())
promesas_mes_liv.columns = ['Division','NumeroPromesas','SumaPromesas']
promesas_mes_liv.fillna(0,inplace=True)
promesas_mes_liv

,Division,NumeroPromesas,SumaPromesas
0,COBRANZA,4,31128.28
1,DKN00REM,75,327977.87
2,DKN02REM,63,374182.44
3,DKN04REM,38,149896.37
4,SA001REM,75,123798.64
5,SA002REM,7,14273.00
6,All,262,1021256.60


In [84]:
#Obtenemos las promesas restantes a partir del día de hoy
restantes_liv = consolidado_pagos_liv.loc[consolidado_pagos_liv['fecha_promesa']>=ayer]
promesas_restantes_liv = pd.pivot_table(restantes_liv,index=['id'],values=['monto_promesa'],aggfunc=[np.sum],dropna=False)
promesas_restantes_liv = pd.DataFrame(promesas_restantes_liv.to_records())
promesas_restantes_liv.columns = ['Division','PromesasRestantes']
promesas_mes_liv.fillna(0,inplace=True)
promesas_restantes_liv

,Division,PromesasRestantes


In [85]:
#Total pagos Liver registrados
pagos_totales_liv = pd.pivot_table(pagos_liv3,index=['Id'],values=['Monto','PagoArancel'],aggfunc=[np.sum,'count'],dropna=False)
pagos_totales_liv = pd.DataFrame(pagos_totales_liv.to_records())
pagos_totales_liv = pagos_totales_liv.iloc[:,[0,3,1,2]]
pagos_totales_liv.columns = ['Division','NumeroPagos','SumaPagos','SumaFacturacion']
pagos_totales_liv.fillna(0,inplace=True)
pagos_totales_liv['FechaFact'] = max(pagos_liv2['Fecha Pago'])
pagos_totales_liv['FechaFact'] = pd.to_datetime(pagos_totales_liv['FechaFact'],dayfirst=True)
pagos_totales_liv

,Division,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact
0,DKN00REM,39,181816.00,50908.4800,2021-01-10
1,DKN02REM,34,205398.99,82159.5960,2021-01-10
2,DKN04REM,19,104195.37,52097.6850,2021-01-10
3,SA001REM,39,62743.15,9411.4725,2021-01-10
4,SA002REM,3,2411.00,385.7600,2021-01-10


In [86]:
final_liv = pd.merge(pagos_totales_liv,promesas_mes_liv,how='left',on=['Division'])
final_liv = pd.merge(final_liv,promesas_restantes_liv,how='left',on=['Division'])
final_liv.insert(loc=0,column='Cliente',value='Liverpool')
final_liv

,Cliente,Division,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact,NumeroPromesas,SumaPromesas,PromesasRestantes
0,Liverpool,DKN00REM,39,181816.00,50908.4800,2021-01-10,75,327977.87,NaN
1,Liverpool,DKN02REM,34,205398.99,82159.5960,2021-01-10,63,374182.44,NaN
2,Liverpool,DKN04REM,19,104195.37,52097.6850,2021-01-10,38,149896.37,NaN
3,Liverpool,SA001REM,39,62743.15,9411.4725,2021-01-10,75,123798.64,NaN
4,Liverpool,SA002REM,3,2411.00,385.7600,2021-01-10,7,14273.00,NaN


# Reporte Bradesco

In [87]:
asignacion_brad_c0 = pd.read_excel(''+ruta_pagos+'/Bradesco/Asignacion_c0/ASIGNACION C0 '+mes2+'.xlsx')
asignacion_brad_c0 = asignacion_brad_c0.iloc[:,[0,2,3,8,10,11,14,25]]
asignacion_brad_extra = pd.read_csv(''+ruta_pagos+'/Bradesco/Asignacion_extra/CSV/Asignacion Extra '+mes2+'.csv',encoding= 'unicode_escape')
asignacion_brad_extra = asignacion_brad_extra.iloc[:,[0,2,3,5,9,25,26,28]]
asignacion_brad_extra.columns = ['CODIGO_ORG','NUMERO_CUENTA','NUMERO_TARJETA','RFC','ESTADO','SALDO_TOTAL','PAGO_DESCUENTO','PAGOSVENCIDOS']

asignacion_brad = pd.concat([asignacion_brad_c0,asignacion_brad_extra],ignore_index=True)

In [88]:
#Hacemos la consulta referente a las gestiones de Bradesco
gestion_brad_mes = cs.query_naucalpan('bradescard_rpt_gestiones_detallado',mes,today)
gestion_brad_mes.columns = ['folio_gestion','firma_id','unegocio_id','NUMERO_TARJETA','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_brad_mes = gestion_brad_mes.loc[gestion_brad_mes['dictamen']=='PROMESA']
gestion_brad_mes['NUMERO_TARJETA'] = pd.to_numeric(gestion_brad_mes['NUMERO_TARJETA'])
gestion_brad_mes['monto_promesa'] = pd.to_numeric(gestion_brad_mes['monto_promesa'])
gestion_brad_mes = gestion_brad_mes.drop_duplicates(subset=['NUMERO_TARJETA','usuario','fecha_promesa','monto_promesa'])

In [89]:
#Hacemos la union de asignacion y promesas
consolidado_brad_mes = pd.merge(gestion_brad_mes,asignacion_brad,how='left',on=['NUMERO_TARJETA'])
cols = ['monto_promesa']
consolidado_brad_mes[cols] = consolidado_brad_mes[cols].apply(pd.to_numeric, errors='coerce')
consolidado_brad_mes = consolidado_brad_mes[(consolidado_brad_mes['monto_promesa']>50)]
consolidado_brad_mes = consolidado_brad_mes[(consolidado_brad_mes['monto_promesa']<500000)]
#consolidado_brad_mes.fillna('COBRANZA',inplace=True)
consolidado_brad_mes['NUMERO_TARJETA'] = pd.to_numeric(consolidado_brad_mes['NUMERO_TARJETA'])
consolidado_brad_mes.head(1)

,folio_gestion,firma_id,unegocio_id,NUMERO_TARJETA,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,...,fecha_promesa,monto_promesa,comentarios,CODIGO_ORG,NUMERO_CUENTA,ESTADO,RFC,SALDO_TOTAL,PAGO_DESCUENTO,PAGOSVENCIDOS
0,9595300,1,3,5063291625426018,ROMERO DELGADO ROGELIO PABLO,8119910367,celular,2021-01-11 16:48:02,VEGAHENA,VEGA HERNANDEZ NATALIE,...,2021-01-25,615.0,8119910367 TT REALIZARA PAGO APRCIAL POR $615 ...,PLCC,5063291625426000,NUEVO LEON,RODR760628Q5801,1889.64,1229.0,C0 5PV


In [90]:
#Obtenemos los pagos totales por division
pagos_totales_brad = pd.pivot_table(pagos_brad2,index=['TIPO_ASIGANCION'],values=['MONTO'],aggfunc=[np.sum,'count'],margins=True,dropna=False)
pagos_totales_brad = pd.DataFrame(pagos_totales_brad.to_records())
pagos_totales_brad = pagos_totales_brad.iloc[:,[0,2,1]]
pagos_totales_brad.columns = ['Division','NumeroPagos','SumaPagos']
pagos_totales_brad.fillna(0,inplace=True)

In [91]:
C04PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 4PV', 'SumaPagos'].item()
C05PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 5PV', 'SumaPagos'].item()
C06PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 6PV', 'SumaPagos'].item()
A = pagos_totales_brad.loc[pagos_totales_brad['Division']=='A', 'SumaPagos'].item()
C1 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C1', 'SumaPagos'].item()
C2 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C2', 'SumaPagos'].item()
C3 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C3', 'SumaPagos'].item()
C4 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C4', 'SumaPagos'].item()
B = pagos_totales_brad.loc[pagos_totales_brad['Division']=='B', 'SumaPagos'].item()

In [92]:
def func(row):
    if row['TIPO_ASIGANCION'] == 'C0 4PV':
        return np.where((C04PV>0) & (C04PV<=c04pv1),arancel_4pv1,
                        np.where((C04PV>c04pv1) & (C04PV<=c04pv2),arancel_4pv2,
                                 np.where((C04PV>c04pv2) & (C04PV<=c04pv3),arancel_4pv3,arancel_4pv3+.03)))
    elif row['TIPO_ASIGANCION'] == 'C0 5PV':
        return np.where((C05PV>0) & (C05PV<=c05pv1),arancel_5pv1,
                        np.where((C05PV>c05pv1) & (C05PV<=c05pv2),arancel_5pv2,
                                 np.where((C05PV>c05pv2) & (C05PV<=c05pv3),arancel_5pv3,arancel_5pv3+.03)))
    elif row['TIPO_ASIGANCION'] == 'C0 6PV':
        return np.where((C06PV>0) & (C06PV<=c06pv1),arancel_6pv1,
                        np.where((C06PV>c06pv1) & (C06PV<=c06pv2),arancel_6pv2,
                                 np.where((C06PV>c06pv2) & (C06PV<=c06pv3),arancel_6pv3,arancel_6pv3+.03)))
    elif row['TIPO_ASIGANCION'] == 'A':
        return np.where((A>0) & (A<=a1),arancel_a1,
                        np.where((A>a1) & (A<=a2),arancel_a2,
                                 np.where((A>a2) & (A<=a3),arancel_a3,arancel_a3+.03)))
    elif row['TIPO_ASIGANCION'] == 'B':
        return np.where((B>0) & (B<=b1),arancel_b1,
                        np.where((B>b1) & (B<=b2),arancel_b2,
                                 np.where((B>b2) & (B<=b3),arancel_b3,arancel_b3+.03)))
    elif row['TIPO_ASIGANCION'] == 'C1':
        return np.where((C1>0) & (C1<=c11),arancel_c11,
                        np.where((C1>c11) & (C1<=c12),arancel_c12,
                                 np.where((C1>c12) & (C1<=c13),arancel_c13,arancel_c13+.03)))
    elif row['TIPO_ASIGANCION'] == 'C2':
        return np.where((C2>0) & (C2<=c21),arancel_c21,
                        np.where((C2>c21) & (C2<=c22),arancel_c22,
                                 np.where((C2>c22) & (C2<=c23),arancel_c23,arancel_c23+.03)))
    elif row['TIPO_ASIGANCION'] == 'C3':
        return np.where((C3>0) & (C3<=c31),arancel_c31,
                        np.where((C3>c31) & (C3<=c32),arancel_c32,
                                 np.where((C3>c32) & (C3<=c33),arancel_c33,arancel_c33+.03)))
    elif row['TIPO_ASIGANCION'] == 'C4':
        return np.where((C4>0) & (C4<=c41),arancel_c41,
                        np.where((C4>c41) & (C4<=c42),arancel_c42,
                                 np.where((C4>c42) & (C4<=c43),arancel_c43,arancel_c43+.03)))
    
    else:
        return 0

pagos_brad2['Arancel'] = pagos_brad2.apply(func, axis=1)

In [93]:
pagos_brad2['Facturacion'] = [x*y for x,y in zip(pagos_brad2['MONTO'],pagos_brad2['Arancel'])]

In [94]:
#Hacemos la union consolidado y pagos
consoliado_pagos_brad = pd.merge(consolidado_brad_mes,pagos_brad2,how='left',left_on=['NUMERO_TARJETA'],right_on=['NUMERO_TARJETA'])
try: consoliado_pagos_brad['fecha_promesa'] = pd.to_datetime(consoliado_pagos_brad['fecha_promesa'],dayfirst=False)
except: consoliado_pagos_brad['fecha_promesa'] = consoliado_pagos_brad['fecha_gestion']

In [95]:
#Condicionamos a los gestores cuyas promesas y pagos estan en un plazo de 3 dias
consoliado_pagos_brad['PagoUnico'] = [np.where((z < x+pd.Timedelta(4, unit='D')) & ((z > x-pd.Timedelta(3, unit='D') )), y, 0) for x,y,z in zip(consoliado_pagos_brad['fecha_promesa'],consoliado_pagos_brad['MONTO'],consoliado_pagos_brad['FECHA_PAGO'])]
consoliado_pagos_brad['PagoUnico'] = [x.item() for x in consoliado_pagos_brad['PagoUnico']]

In [96]:
consoliado_pagos_brad['PagoReal'] = [x*y for x,y in zip(consoliado_pagos_brad['PagoUnico'],consoliado_pagos_brad['Arancel'])]
consoliado_pagos_brad2 = consoliado_pagos_brad.drop_duplicates(subset=['NUMERO_CUENTA','MONTO','FECHA_PAGO','usuario'])

In [97]:
#Obtenemos datos de las promesas de Bradesco
prom_brad_div_mes = pd.pivot_table(consoliado_pagos_brad2,index='PAGOSVENCIDOS',values=['monto_promesa'], aggfunc = ['count',np.sum], margins=True,dropna=False)
prom_brad_div_mes = pd.DataFrame(prom_brad_div_mes.to_records())
prom_brad_div_mes.columns = ['Division','NumeroPromesas','SumaPromesas']
prom_brad_div_mes.fillna(0,inplace=True)
prom_brad_div_mes

,Division,NumeroPromesas,SumaPromesas
0,A,17,38197.00
1,B,5,9797.00
2,C0 4PV,142,237789.09
3,C0 5PV,76,125807.00
4,C0 6PV,55,111773.00
5,C1,81,132339.00
6,C2,79,106979.00
7,C3,68,92581.00
8,C4,24,27318.00
9,All,547,882580.09


In [98]:
#Obtenemos las promesas restantes a partir del día de hoy
restantes_brad = consoliado_pagos_brad2.loc[consoliado_pagos_brad2['fecha_promesa']>=ayer]
promesas_restantes_brad = pd.pivot_table(restantes_brad,index=['PAGOSVENCIDOS'],values=['monto_promesa'],aggfunc=[np.sum],dropna=False)
promesas_restantes_brad = pd.DataFrame(promesas_restantes_brad.to_records())
promesas_restantes_brad.columns = ['Division','PromesasRestantes']
promesas_restantes_brad.fillna(0,inplace=True)
promesas_restantes_brad

,Division,PromesasRestantes
0,A,5105.00
1,B,1050.00
2,C0 4PV,136473.09
3,C0 5PV,78999.00
4,C0 6PV,63407.00
5,C1,46004.00
6,C2,59896.00
7,C3,44631.00
8,C4,9738.00


In [99]:
pagos_totales_brad2 = pd.pivot_table(pagos_brad2,index=['TIPO_ASIGANCION'],values=['MONTO','Facturacion'],aggfunc=['count',np.sum],dropna=False)
pagos_totales_brad2 = pd.DataFrame(pagos_totales_brad2.to_records())
pagos_totales_brad2 = pagos_totales_brad2.iloc[:,[0,1,4,3]]
pagos_totales_brad2.columns = ['Division','NumeroPagos','SumaPagos','SumaFacturacion']
pagos_totales_brad2.fillna(0,inplace=True)
pagos_totales_brad2['FechaFact'] = max(pagos_brad2['FECHA_PAGO'])

In [100]:
final_brad = pd.merge(pagos_totales_brad2,prom_brad_div_mes,how='left',on=['Division'])
final_brad = pd.merge(final_brad,promesas_restantes_brad,how='left',on=['Division'])
final_brad.fillna(0,inplace=True)
final_brad.insert(loc=0,column='Cliente',value='Bradesco')
final_brad

,Cliente,Division,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact,NumeroPromesas,SumaPromesas,PromesasRestantes
0,Bradesco,A,4,7250.00,1957.5000,2021-01-10,17,38197.00,5105.00
1,Bradesco,B,3,4797.00,1199.2500,2021-01-10,5,9797.00,1050.00
2,Bradesco,C0 4PV,95,166763.13,11673.4191,2021-01-10,142,237789.09,136473.09
3,Bradesco,C0 5PV,61,84299.10,7586.9190,2021-01-10,76,125807.00,78999.00
4,Bradesco,C0 6PV,37,62408.00,6240.8000,2021-01-10,55,111773.00,63407.00
5,Bradesco,C1,53,77593.00,13190.8100,2021-01-10,81,132339.00,46004.00
6,Bradesco,C2,33,49080.84,9325.3596,2021-01-10,79,106979.00,59896.00
7,Bradesco,C3,20,32240.00,6770.4000,2021-01-10,68,92581.00,44631.00
8,Bradesco,C4,6,4471.00,1028.3300,2021-01-10,24,27318.00,9738.00


# Reporte Banco Azteca  

In [101]:
# #Hacemos la consulta referente a las gestiones de Banco Azteca
# sql_gest_baz = cursor.callproc('baz_rpt_gestiones_detallado', [mes,today])
# for result in cursor.stored_results():
#      gestion_baz = pd.DataFrame(result.fetchall())
# gestion_baz.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion_resultado','fecha_promesa','monto_promesa','comentarios']
# gestion_baz = gestion_baz.loc[gestion_baz['dictamen']=='PROMESA']
# gestion_baz['monto_promesa'] = pd.to_numeric(gestion_baz['monto_promesa'])
# gestion_baz = gestion_baz.loc[gestion_baz['fecha_promesa'].map(str).str[:4].map(int)<2100]
# gestion_baz['fecha_promesa'] = pd.to_datetime(gestion_baz['fecha_promesa'],dayfirst=False)
# #gestion_baz.head(3)

In [102]:
# #Obtenemos datos de las promesas de Banco Azteca
# gestion_baz = gestion_baz.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
# prom_baz_div_mes = pd.pivot_table(gestion_baz,index=['dictamen'],values=['unegocio_id','monto_promesa'],aggfunc=('count',np.sum))
# prom_baz_div_mes = pd.DataFrame(prom_baz_div_mes.to_records())
# prom_baz_div_mes = prom_baz_div_mes.iloc[:,[0,1,2]]
# prom_baz_div_mes.columns = ['Division','NumeroPromesas','SumaPromesas']
# prom_baz_div_mes.fillna(0,inplace=True)
# prom_baz_div_mes = prom_baz_div_mes.replace('PROMESA','COBRANZA')
# prom_baz_div_mes

In [103]:
# #Obtenemos las promesas restantes a partir del día de hoy
# restantes_baz = gestion_baz.loc[gestion_baz['fecha_promesa']>=ayer]

# promesas_restantes_baz = pd.pivot_table(restantes_baz,index=['dictamen'],values=['monto_promesa'],aggfunc=[np.sum],dropna=False)
# promesas_restantes_baz = pd.DataFrame(promesas_restantes_baz.to_records())
# promesas_restantes_baz.columns = ['Division','PromesasRestantes']
# promesas_restantes_baz = promesas_restantes_baz.replace('PROMESA','COBRANZA')
# promesas_restantes_baz.fillna(0,inplace=True)
# promesas_restantes_baz

In [104]:
# pagos_baz2['division'] = 'COBRANZA'
# pagos_baz2['FNFACTURACION'] = [x*.2 for x in pagos_baz2['FNMONTOPAGADO']]
# pagos_totales_baz = pd.pivot_table(pagos_baz2,index=['division'],values=['FNMONTOPAGADO'],aggfunc=['count',np.sum],dropna=False)
# pagos_totales_baz = pd.DataFrame(pagos_totales_baz.to_records())
# #pagos_totales_baz = pagos_totales_baz.iloc[:,[0,1,4,3]]
# pagos_totales_baz.columns = ['Division','NumeroPagos','SumaPagos']
# #pagos_totales_baz.fillna(0,inplace=True)
# #pagos_totales_baz['FechaFact'] = ayer
# pagos_totales_baz

In [105]:
# fact_baz['Total Recuperado'] = np.where(fact_baz['Inicio de Semana']<mes3,fact_baz['Total Recuperado']*.85,fact_baz['Total Recuperado'])
# fact_baz['Total Comisión'] = np.where(fact_baz['Inicio de Semana']<mes3,fact_baz['Total Comisión']*.85,fact_baz['Total Comisión'])

In [106]:
# #Hacemos la consulta de pagos y facturacion
# fact_baz['Division2'] = 'COBRANZA'
# pagos_fact_baz = pd.pivot_table(fact_baz,index=['Division2'],values=['Total Recuperado','Total Comisión'],aggfunc=[np.sum])
# pagos_fact_baz = to_dataframe(pagos_fact_baz)
# pagos_fact_baz = pagos_fact_baz.iloc[:,[0,2,1]]
# pagos_fact_baz.columns = ['Division','SumaPagos','SumaFacturacion']
# pagos_fact_baz['FechaFact'] = max(fact_baz['Final de Semana'])
# pagos_fact_baz

In [107]:
# final_baz = pd.merge(pagos_fact_baz,prom_baz_div_mes,how='left',on=['Division'])
# final_baz = pd.merge(final_baz,promesas_restantes_baz,how='left',on=['Division'])
# final_baz.fillna(0,inplace=True)
# final_baz.insert(loc=0,column='Cliente',value='Banco Azteca')
# final_baz

# Reporte Credifiel

In [108]:
#Hacemos la consulta referente a las gestiones de Credifiel
gestion_cred = cs.query_naucalpan('credifiel_rpt_gestiones_detallado',mes,today)
gestion_cred.columns = ['folio_gestion','unegocio_id','fecha_gestion','hora_gestion','credito','nombre_credito','telefono','usuario','nombre_usuario','accion','resultado','fecha_promesa','monto_promesa','comentarios','accion_credifiel','resultado_credifiel','dictamen']
gestion_cred['monto_promesa'] = pd.to_numeric(gestion_cred['monto_promesa'])
gestion_cred = gestion_cred.loc[gestion_cred['accion_credifiel']=='PDP']
gestion_cred['credito'] = [x[3:] for x in gestion_cred['credito']]
gestion_cred['credito'] = pd.to_numeric(gestion_cred['credito'])
#gestion_cred.head(3)

In [109]:
#Hacemos la consulta de la asignacion
asignacion_cred = pd.read_excel('/home/estadistico/Documents/Erick/Credifiel/Asignacion/Asignacion Credifiel '+mes2+'.xlsx')
asignacion_cred = asignacion_cred.drop_duplicates(subset=['CREDITO','SALDO TOTAL REAL'])
asignacion_cred['CREDITO'] = pd.to_numeric(asignacion_cred['CREDITO'])

In [110]:
#Hacemos el consolidado de la asignacion y promesado
consolidado_cred = pd.merge(gestion_cred,asignacion_cred,how='left',left_on=['credito'],right_on=['CREDITO'])
cols = ['monto_promesa','unegocio_id']
consolidado_cred[cols] = consolidado_cred[cols].apply(pd.to_numeric, errors='coerce')
consolidado_cred = consolidado_cred.loc[consolidado_cred['monto_promesa']>50]
consolidado_cred = consolidado_cred[(consolidado_cred['monto_promesa']<500000)]
consolidado_cred['fecha_promesa'] = pd.to_datetime(consolidado_cred['fecha_promesa'],dayfirst=False)
consolidado_cred.replace('CONTACTO','EXTRA',inplace=True)

In [111]:
pagos_cred3 = pd.merge(pagos_cred2,consolidado_cred[['credito','nombre_usuario','fecha_promesa','monto_promesa','ARANCEL']],how='left',left_on=['ID'],right_on=['credito'])
pagos_cred3['ARANCEL'].fillna(np.mean(pagos_cred3['ARANCEL']),inplace=True)
pagos_cred3['PagoFacturacion'] = [x*y for x,y in zip(pagos_cred3['ARANCEL'],pagos_cred3['IMPORTE'])]
pagos_cred3['division'] = 'EXTRA'
pagos_cred3 = pagos_cred3.drop_duplicates(subset=['ID','IMPORTE'])

In [112]:
#Obtenemos datos de las promesas de Bradesco
prom_cred_div_mes = pd.pivot_table(consolidado_cred,index=['dictamen'],values=['monto_promesa'], aggfunc=['count',np.sum], dropna=False)
prom_cred_div_mes = pd.DataFrame(prom_cred_div_mes.to_records())
#prom_cred_div_mes = prom_cred_div_mes.iloc[:,[0,4,8,6,5]]
prom_cred_div_mes.columns = ['Division','NumeroPromesas','SumaPromesas']
#prom_cred_div_mes = prom_cred_div_mes.sort_values(by=['SumaPago'],ascending =False)
#prom_cred_div_mes = prom_cred_div_mes.reset_index(drop=True)
#prom_cred_div_mes.fillna(0,inplace=True)
#prom_brad_div_mes[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = prom_brad_gestor_mes[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
prom_cred_div_mes

,Division,NumeroPromesas,SumaPromesas
0,EXTRA,37,136948.34


In [113]:
#Obtenemos las promesas restantes a partir del día de hoy
restantes_cred = consolidado_cred.loc[consolidado_cred['fecha_promesa']>=ayer]

promesas_restantes_cred = pd.pivot_table(restantes_cred,index=['dictamen'],values=['monto_promesa'],aggfunc=[np.sum],dropna=False)
promesas_restantes_cred = pd.DataFrame(promesas_restantes_cred.to_records())
#promesas_restantes_brad = promesas_restantes_brad.iloc[:,[0,3,8,6,5]]
promesas_restantes_cred.columns = ['Division','PromesasRestantes']
promesas_restantes_cred.fillna(0,inplace=True)

#posible cambio
#promesas_restantes_brad = promesas_restantes_brad.sort_values(by='SumaPagos',ascending=False)
#promesas_restantes_brad[['SumaPromesas']] = promesas_restantes_brad[['SumaPromesas']].applymap("{0:.2f}".format)
promesas_restantes_cred

,Division,PromesasRestantes
0,EXTRA,94844.23


In [114]:
pagos_totales_cred = pd.pivot_table(pagos_cred3,index=['division'],values=['IMPORTE','PagoFacturacion'],aggfunc=['count',np.sum],dropna=False)
pagos_totales_cred = pd.DataFrame(pagos_totales_cred.to_records())
pagos_totales_cred = pagos_totales_cred.iloc[:,[0,1,3,4]]
pagos_totales_cred.columns = ['Division','NumeroPagos','SumaPagos','SumaFacturacion']
pagos_totales_cred.fillna(0,inplace=True)
#pagos_totales_cred[['SumaPagos','SumaFacturacion']] = pagos_totales_cred[['SumaPagos','SumaFacturacion']].applymap("${0:,.2f}".format)
pagos_totales_cred['FechaFact'] = max(pagos_cred3['FECHA'])
pagos_totales_cred

,Division,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact
0,EXTRA,21,107727.5,23762.778333,2021-01-08


In [115]:
final_cred = pd.merge(pagos_totales_cred,prom_cred_div_mes,how='left',on=['Division'])
final_cred = pd.merge(final_cred,promesas_restantes_cred,how='left',on=['Division'])
final_cred.fillna(0,inplace=True)
final_cred.insert(loc=0,column='Cliente',value='Credifiel')
final_cred = final_cred.replace('EXTRA','COBRANZA')
final_cred

,Cliente,Division,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact,NumeroPromesas,SumaPromesas,PromesasRestantes
0,Credifiel,COBRANZA,21,107727.5,23762.778333,2021-01-08,37,136948.34,94844.23


# Reporte final Naucalpan

In [116]:
final_naucalpan = pd.concat([final_liv,final_brad],ignore_index=True)
final_naucalpan = pd.concat([final_naucalpan,final_cred],ignore_index=True)
#final_naucalpan = pd.concat([final_naucalpan,final_baz],ignore_index=True)
final_naucalpan.insert(loc=0,column='Sucursal',value='Naucalpan')
final_naucalpan.insert(loc=0,column='Pais',value='Mexico')
final_naucalpan['TendenciaCierre1'] = [(30/x.day)*y for x,y in zip(final_naucalpan['FechaFact'],final_naucalpan['SumaFacturacion'])]
final_naucalpan = final_naucalpan.replace('EXTRA','EXTRAJUDICIAL')
final_naucalpan = final_naucalpan.replace('COBRANZA','EXTRAJUDICIAL')
final_naucalpan.fillna('',inplace=True)
final_naucalpan

,Pais,Sucursal,Cliente,Division,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact,NumeroPromesas,SumaPromesas,PromesasRestantes,TendenciaCierre1
0,Mexico,Naucalpan,Liverpool,DKN00REM,39,181816.00,50908.480000,2021-01-10,75,327977.87,,152725.44000
1,Mexico,Naucalpan,Liverpool,DKN02REM,34,205398.99,82159.596000,2021-01-10,63,374182.44,,246478.78800
2,Mexico,Naucalpan,Liverpool,DKN04REM,19,104195.37,52097.685000,2021-01-10,38,149896.37,,156293.05500
3,Mexico,Naucalpan,Liverpool,SA001REM,39,62743.15,9411.472500,2021-01-10,75,123798.64,,28234.41750
4,Mexico,Naucalpan,Liverpool,SA002REM,3,2411.00,385.760000,2021-01-10,7,14273.00,,1157.28000
5,Mexico,Naucalpan,Bradesco,A,4,7250.00,1957.500000,2021-01-10,17,38197.00,5105,5872.50000
6,Mexico,Naucalpan,Bradesco,B,3,4797.00,1199.250000,2021-01-10,5,9797.00,1050,3597.75000
7,Mexico,Naucalpan,Bradesco,C0 4PV,95,166763.13,11673.419100,2021-01-10,142,237789.09,136473,35020.25730
8,Mexico,Naucalpan,Bradesco,C0 5PV,61,84299.10,7586.919000,2021-01-10,76,125807.00,78999,22760.75700
9,Mexico,Naucalpan,Bradesco,C0 6PV,37,62408.00,6240.800000,2021-01-10,55,111773.00,63407,18722.40000


# Pagos sin promesa asignada

In [117]:
#Pagos de bradesco que no tienen promesa asignada
gestion_brad_mes2 = gestion_brad_mes.drop_duplicates(subset='NUMERO_TARJETA')
pagos_noasig_brad = pd.merge(pagos_brad2,gestion_brad_mes2[['NUMERO_TARJETA','nombre_credito']],how='left',on='NUMERO_TARJETA')
pagos_noasig_brad = pagos_noasig_brad.loc[pagos_noasig_brad['nombre_credito'].isnull()]
pagos_noasig_brad_pivot = pd.pivot_table(pagos_noasig_brad,index=['TIPO_ASIGANCION'],values=['MONTO'],aggfunc=[np.sum,'count'])
pagos_noasig_brad_pivot = pd.DataFrame(pagos_noasig_brad_pivot.to_records())
pagos_noasig_brad_pivot.columns = ['Division','RecuperadoNoPromesa','PagosNoPromesa']
pagos_noasig_brad_pivot

,Division,RecuperadoNoPromesa,PagosNoPromesa
0,B,1500.00,1
1,C0 4PV,74044.13,53
2,C0 5PV,45745.08,35
3,C0 6PV,28605.00,19
4,C1,36892.00,27
5,C2,22649.84,15
6,C3,2222.00,2


In [118]:
#Pagos de Liverpool que no tienen promesa asignada
gestion_liv_mes2 = gestion_liv_mes.drop_duplicates(subset='credito')
pagos_noasig_liv = pd.merge(pagos_liv2,gestion_liv_mes2[['credito','nombre_credito']],left_on='Cuenta',right_on='credito',how='left')
pagos_noasig_liv = pagos_noasig_liv.loc[pagos_noasig_liv['nombre_credito'].isnull()]
pagos_noasig_liv_pivot = pd.pivot_table(pagos_noasig_liv,index=['Id'],values=['Monto'],aggfunc=[np.sum,'count'])
pagos_noasig_liv_pivot = pd.DataFrame(pagos_noasig_liv_pivot.to_records())
pagos_noasig_liv_pivot.columns = ['Division','RecuperadoNoPromesa','PagosNoPromesa']
pagos_noasig_liv_pivot

,Division,RecuperadoNoPromesa,PagosNoPromesa
0,DKN00REM,35602.00,11
1,DKN02REM,44173.99,19
2,DKN04REM,26263.00,5
3,SA001REM,13979.00,11
4,SA002REM,2411.00,3


In [119]:
#Pagos de Liverpool que no tienen promesa asignada
gestion_cred2 = gestion_cred.drop_duplicates(subset='credito')
pagos_noasig_cred = pd.merge(pagos_cred2,gestion_cred2[['credito','nombre_credito']],left_on='ID',right_on='credito',how='left')
pagos_noasig_cred = pagos_noasig_cred.loc[pagos_noasig_cred['nombre_credito'].isnull()]
pagos_noasig_cred['Cliente'] = 'EXTRAJUDICIAL'
pagos_noasig_cred_pivot = pd.pivot_table(pagos_noasig_cred,index=['Cliente'],values=['IMPORTE'],aggfunc=[np.sum,'count'])
pagos_noasig_cred_pivot = pd.DataFrame(pagos_noasig_cred_pivot.to_records())
pagos_noasig_cred_pivot.columns = ['Division','RecuperadoNoPromesa','PagosNoPromesa']
pagos_noasig_cred_pivot

,Division,RecuperadoNoPromesa,PagosNoPromesa
0,EXTRAJUDICIAL,62454.0,8


In [120]:
list1 = [pagos_noasig_cred_pivot,pagos_noasig_liv_pivot,pagos_noasig_brad_pivot]
pagos_noasig_naucalpan = pd.concat(list1,ignore_index=True)
pagos_noasig_naucalpan

,Division,RecuperadoNoPromesa,PagosNoPromesa
0,EXTRAJUDICIAL,62454.00,8
1,DKN00REM,35602.00,11
2,DKN02REM,44173.99,19
3,DKN04REM,26263.00,5
4,SA001REM,13979.00,11
5,SA002REM,2411.00,3
6,B,1500.00,1
7,C0 4PV,74044.13,53
8,C0 5PV,45745.08,35
9,C0 6PV,28605.00,19


In [121]:
final = pd.merge(final_naucalpan,pagos_noasig_naucalpan,on='Division',how='left')
final

,Pais,Sucursal,Cliente,Division,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact,NumeroPromesas,SumaPromesas,PromesasRestantes,TendenciaCierre1,RecuperadoNoPromesa,PagosNoPromesa
0,Mexico,Naucalpan,Liverpool,DKN00REM,39,181816.00,50908.480000,2021-01-10,75,327977.87,,152725.44000,35602.00,11.0
1,Mexico,Naucalpan,Liverpool,DKN02REM,34,205398.99,82159.596000,2021-01-10,63,374182.44,,246478.78800,44173.99,19.0
2,Mexico,Naucalpan,Liverpool,DKN04REM,19,104195.37,52097.685000,2021-01-10,38,149896.37,,156293.05500,26263.00,5.0
3,Mexico,Naucalpan,Liverpool,SA001REM,39,62743.15,9411.472500,2021-01-10,75,123798.64,,28234.41750,13979.00,11.0
4,Mexico,Naucalpan,Liverpool,SA002REM,3,2411.00,385.760000,2021-01-10,7,14273.00,,1157.28000,2411.00,3.0
5,Mexico,Naucalpan,Bradesco,A,4,7250.00,1957.500000,2021-01-10,17,38197.00,5105,5872.50000,NaN,NaN
6,Mexico,Naucalpan,Bradesco,B,3,4797.00,1199.250000,2021-01-10,5,9797.00,1050,3597.75000,1500.00,1.0
7,Mexico,Naucalpan,Bradesco,C0 4PV,95,166763.13,11673.419100,2021-01-10,142,237789.09,136473,35020.25730,74044.13,53.0
8,Mexico,Naucalpan,Bradesco,C0 5PV,61,84299.10,7586.919000,2021-01-10,76,125807.00,78999,22760.75700,45745.08,35.0
9,Mexico,Naucalpan,Bradesco,C0 6PV,37,62408.00,6240.800000,2021-01-10,55,111773.00,63407,18722.40000,28605.00,19.0


In [122]:
final.to_csv(''+ruta_pagos+'/'+mes2+'/Reportes diarios/Reporte sucursal/Reporte sucursal '+today2+'.csv',index=False,header=True)